In [1]:
import sys
import os
# navigate two directories up
dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(dir)
# print working directory
# mavigate to the top of working directory
os.chdir(dir)
print(os.getcwd())
from repomanager.statemanager.vspace.vsearch import VectorSearch
import pickle

/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager


/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pwd

'/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager'

In [3]:
sys.path

['/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/repomanager/tests/documentation_vector_search',
 '/Users/chinmayshrivastava/anaconda3/lib/python311.zip',
 '/Users/chinmayshrivastava/anaconda3/lib/python3.11',
 '/Users/chinmayshrivastava/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/venv/lib/python3.11/site-packages',
 '/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager']

In [4]:
repo_id = "45526e5b-f544-4016-8381-f88f5ca095ea"

In [5]:
G = pickle.load(open("repomanager/statemanager/state/{}/state_0.pkl".format(repo_id), "rb"))
print(G)

DiGraph with 31986 nodes and 53803 edges


In [6]:
vs = VectorSearch(
    collection_name="docs",
    collection_path="repomanager/statemanager/state/{}/meta/storage".format(repo_id),
)

In [7]:
vs.collection.count()

14163

In [8]:
query = """For the user query, match the best use case or insight from a code base.
user query: How to rerank"""
nodes = [x["node"] for x in vs.search_all(query)['metadatas'][0]]
distances = vs.search_all(query)['distances'][0]
distances

[0.16178828477859497,
 0.1697811484336853,
 0.1707085371017456,
 0.17160683870315552,
 0.17160683870315552]

In [9]:
i = 0
for node in nodes:
    # get the node from the graph and get the content
    print(G.nodes[node]["content"])
    # print(G.nodes[node[:len(node)-1]+f"{int(node[-1])-1}"]["content"])
    i = i + 1
    if i == 3:
        break

Define Advanced Retriever
We define an advanced retriever that performs the following steps:

Query generation/rewriting: generate multiple queries given the original user query
Perform retrieval for each query over an ensemble of retrievers.
Reranking/fusion: fuse results from all queries, and apply a reranking step to "fuse" the top relevant results!

Then in the next section we'll plug this into our response synthesis module.

Step 1: Query Generation/Rewriting
The first step is to generate queries from the original query to better match the query intent, and increase precision/recall of the retrieved results. For instance, we might be able to rewrite the query into smaller queries.
We can do this by prompting ChatGPT.

input:
```python
from llama_index import PromptTemplate
```
output:
input:
```python
query_str = "How do the models developed in this work compare to open-source chat models based on the benchmarks tested?"
```
output:
input:
```python
query_gen_prompt_str = (
    "Y

In [10]:
def get_docs(
    query: str,
    vs: VectorSearch,
    G,
    top_k: int = 5,
    max_depth: int = 3,
    ):

    nodes = [x["node"] for x in vs.search_all(query, top_k=top_k)['metadatas'][0]]
    docs = []
    for node in nodes:
        # get the node from the graph and get the content
        _nodes = [node]
        for i in range(max_depth):
            _nodes.append(_nodes[i][:len(_nodes[i])-1]+f"{int(_nodes[i][-1])-1}")
            _nodes.append(_nodes[i][:len(_nodes[i])-1]+f"{int(_nodes[i][-1])+1}")
        _nodes = list(set(_nodes))
        _nodes = [x for x in _nodes if x in G.nodes]
        _docs = [G.nodes[x]["content"] for x in _nodes]
        docs.append(_docs)
    return docs


In [11]:
query = """For the user query, match the best use case or insight from a code base.
user query: How to use agents"""
get_docs(query, vs, G, top_k=5, max_depth=3)

[['Build Retriever-Enabled OpenAI Agent\nWe build a top-level agent that can orchestrate across the different document agents to answer any user query.\nThis RetrieverOpenAIAgent performs tool retrieval before tool use (unlike a default agent that tries to put all tools in the prompt).\nImprovements from V0: We make the following improvements compared to the "base" version in V0.\n\nAdding in reranking: we use Cohere reranker to better filter the candidate set of documents.\nAdding in a query planning tool: we add an explicit query planning tool that\'s dynamically created based on the set of retrieved tools.\n\n\ninput:\n```python\n# define tool for each document agent\nall_tools = []\nfor file_base, agent in agents_dict.items():\n    summary = extra_info_dict[file_base]["summary"]\n    doc_tool = QueryEngineTool(\n        query_engine=agent,\n        metadata=ToolMetadata(\n            name=f"tool_{file_base}",\n            description=summary,\n        ),\n    )\n    all_tools.appen

In [12]:
# rerank

import nltk

# tokinize
def tokenize(text):
    return nltk.word_tokenize(text)

# rank the documents according to the tokenized query
def rank_docs(docs, query):
    query_tokens = tokenize(query)
    scores = []
    for doc in docs:
        doc_tokens = tokenize(doc)
        score = 0
        for token in query_tokens:
            score += doc_tokens.count(token)
        scores.append(score)
    reranked_docs = [x for _,x in sorted(zip(scores,docs), reverse=True)]
    return reranked_docs

In [13]:
query = """Find the best matching usecase or insight.
user query: What is fusion retrieval?"""

docs = []
for doc in get_docs(query, vs, G, top_k=5, max_depth=3):
    docs.extend(doc)
docs

['Fuse the Indexes!\nIn this step, we fuse our indexes into a single retriever. This retriever will also generate augment our query by generating extra queries related to the original question, and aggregate the results.\nThis setup will query 4 times, once with your original query, and generate 3 more queries.\nBy default, it uses the following prompt to generate extra queries:\nQUERY_GEN_PROMPT = (\n    "You are a helpful assistant that generates multiple search queries based on a "\n    "single input query. Generate {num_queries} search queries, one on each line, "\n    "related to the following input query:\\n"\n    "Query: {query}\\n"\n    "Queries:\\n"\n)\n\n\ninput:\n```python\nfrom llama_index.retrievers import QueryFusionRetriever\n\nretriever = QueryFusionRetriever(\n    [index_1.as_retriever(), index_2.as_retriever()],\n    similarity_top_k=2,\n    num_queries=4,  # set this to 1 to disable query generation\n    use_async=True,\n    verbose=True,\n    # query_gen_prompt="...

In [14]:
rank_docs(docs, query)

['Fuse the Indexes!\nIn this step, we fuse our indexes into a single retriever. This retriever will also generate augment our query by generating extra queries related to the original question, and aggregate the results.\nThis setup will query 4 times, once with your original query, and generate 3 more queries.\nBy default, it uses the following prompt to generate extra queries:\nQUERY_GEN_PROMPT = (\n    "You are a helpful assistant that generates multiple search queries based on a "\n    "single input query. Generate {num_queries} search queries, one on each line, "\n    "related to the following input query:\\n"\n    "Query: {query}\\n"\n    "Queries:\\n"\n)\n\n\ninput:\n```python\nfrom llama_index.retrievers import QueryFusionRetriever\n\nretriever = QueryFusionRetriever(\n    [index_1.as_retriever(), index_2.as_retriever()],\n    similarity_top_k=2,\n    num_queries=4,  # set this to 1 to disable query generation\n    use_async=True,\n    verbose=True,\n    # query_gen_prompt="...

In [15]:
from repomanager.statemanager.llms.completion import get_llm

llm = get_llm()
llm2 = get_llm(model='gpt-4')

In [19]:
query = 'How to load vector store and graph store together?'
_query = """Find the best matching usecase or insight.
user query: {}""".format(query)
docs = []
for doc in get_docs(_query, vs, G, top_k=5, max_depth=3):
    docs.extend(doc)
reranked_docs = rank_docs(docs, _query)
top_three = reranked_docs[:3]
prompt = """For the user query, use the information attached to answer it in short form.
Provide simple examples (if possible):\n"""
i = 0
for doc in top_three:
    prompt += "Source {}\n".format(i)
    prompt += doc + "\n"
    i += 1

prompt += "Query:\n"
prompt += query + "\n"
prompt += "Answer:\n"
# print the number of words in the prompt
len(prompt.split())

1025

In [20]:
# import display and markdown
from IPython.display import display, Markdown

display(Markdown(llm.complete(prompt).text))

To load a vector store and graph store together, you can use the following code:

```python
from llama_index import VectorStoreIndex, GraphStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext

# Load vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

# Load graph store
graph_store = GraphStoreIndex(graph_collection=graph_collection)
index.add_graph_store(graph_store)
```

This code initializes a vector store using Chroma as the vector store and a graph store using the specified graph collection. The vector store and graph store are then added to the index.

In [21]:
display(Markdown(llm2.complete(prompt).text))

To load a vector store and graph store together, you need to initialize a client, create a collection to store your data, assign the vector store to a StorageContext, and initialize your VectorStoreIndex using that StorageContext. Here is an example using Chroma as the vector store:

```python
import chromadb
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext

# load some documents
documents = SimpleDirectoryReader("./data").load_data()

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

# create a query engine and query
query_engine = index.as_query_engine()
response = query_engine.query("What is the meaning of life?")
print(response)
```

If you've already created and stored your embeddings, you can load them directly without loading your documents or creating a new VectorStoreIndex:

```python
import chromadb
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext

# initialize client
db = chromadb.PersistentClient(path="./chroma_db")

# get collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

# create a query engine
query_engine = index.as_query_engine()
response = query_engine.query("What is llama2?")
print(response)
```

In [157]:
# perform page rank on the graph
import networkx as nx
import numpy as np

page_rank = nx.pagerank(G)
page_rank

{'root': 2.535797987573841e-05,
 'LICENSE': 2.6558362187526002e-05,
 'CHANGELOG.md': 2.6558362187526002e-05,
 '.pre-commit-config.yaml': 2.6558362187526002e-05,
 'Makefile': 2.8612681822994494e-05,
 'CITATION.cff': 2.6558362187526002e-05,
 'pyproject.toml': 2.6558362187526002e-05,
 'README.md': 9.657432073854376e-05,
 'CONTRIBUTING.md': 2.6558362187526002e-05,
 'poetry.lock': 2.6558362187526002e-05,
 '.readthedocs.yaml': 2.6558362187526002e-05,
 'experimental': 2.7223581194882335e-05,
 'splitter_playground': 2.8673215955606596e-05,
 'app.py': 4.9838092699503767e-05,
 'cli': 2.8673215955606596e-05,
 'configuration.py': 3.858561624856383e-05,
 'cli_add.py': 3.0910707139595646e-05,
 '__init__.py': 0.0006499568398591388,
 'cli_init.py': 3.0910707139595646e-05,
 'cli_query.py': 3.0910707139595646e-05,
 '__main__.py': 2.885513885056203e-05,
 'openai_fine_tuning': 2.8673215955606596e-05,
 'openai_fine_tuning.ipynb': 3.586639057649473e-05,
 'launch_training.py': 3.351801748366019e-05,
 'valida

In [162]:
page_rank['OpenAIAgent'], page_rank['AgentRunner'], page_rank["BaseAgentRunner"], page_rank["BaseReader"], page_rank["BaseAgent"]

(4.1214549799908395e-05,
 3.139584137403907e-05,
 2.818730587641715e-05,
 0.00021939831410859982,
 3.34182345009083e-05)

In [164]:
vs2 = VectorSearch(
    collection_name="explanations",
    collection_path="repomanager/statemanager/state/{}/meta/storage".format(repo_id),
)

In [186]:
query = "How to load a pdf"
res = vs2.search(query, top_k=5, type="class")
res

[('PDFReader', 'class'),
 ('DocxReader', 'class'),
 ('BaseReader', 'class'),
 ('GoogleDocsReader', 'class'),
 ('MyScaleReader', 'class')]

In [187]:
# rank res by page rank
res_ranked = sorted(res, key=lambda x: page_rank[x[0]], reverse=True)
res_ranked

[('BaseReader', 'class'),
 ('PDFReader', 'class'),
 ('GoogleDocsReader', 'class'),
 ('DocxReader', 'class'),
 ('MyScaleReader', 'class')]